# collecting data

In [1]:
import pandas as pd

# just the links
seattlewiki="https://en.wikipedia.org/wiki/Seattle"
newyorkwiki="https://en.wikipedia.org/wiki/New_York_City"
bostonwiki="https://en.wikipedia.org/wiki/Boston"

# data frames as lists:

seattletab=pd.read_html(seattlewiki, # link
                        header=0, # where is the header?
                        flavor='bs4', # helper to translate html
                        attrs={'class': 'wikitable'})
newyorktab=pd.read_html(newyorkwiki, # link
                        header=0, # where is the header?
                        flavor='bs4', # helper to translate html
                        attrs={'class': 'wikitable'})
bostontab=pd.read_html(bostonwiki, # link
                        header=0, # where is the header?
                        flavor='bs4', # helper to translate html
                        attrs={'class': 'wikitable'})

Tables needed:

In [2]:
dataSea=seattletab[1].copy()
dataBos=bostontab[2].copy()
dataNY=newyorktab[3].copy()

Verify column names

In [3]:
dataBos.columns

Index(['Race/ethnicity', '2020[146]', '2010[147]', '1990[142]', '1970[142]',
       '1940[142]'],
      dtype='object')

Previous view of changes

In [4]:
# BYE anything that looks like \[.+\], where .+ means one or more character:
dataBos.columns.str.replace('\[.+\]|\s|\/.+','', regex=True)

Index(['Race', '2020', '2010', '1990', '1970', '1940'], dtype='object')

Changing column names:

In [5]:
#Then:
dataSea.columns=dataSea.columns.str.replace('\[.+\]|\s|\/.+','', regex=True)
dataBos.columns=dataBos.columns.str.replace('\[.+\]|\s|\/.+','', regex=True)
dataNY.columns=dataNY.columns.str.replace('\[.+\]|\s|\/.+','', regex=True)

In [6]:
# problem, first column name:
dataSea.columns[0],dataNY.columns[0],dataBos.columns[0]

('Racialcomposition', 'Historicaldemographics', 'Race')

In [7]:
#Are they all the same?
set(dataNY.columns)&set(dataBos.columns)&set(dataSea.columns)

{'1940', '1970', '1990', '2010', '2020'}

In [8]:
(set(dataSea.columns)^set(dataNY.columns))

{'Historicaldemographics', 'Racialcomposition'}

Making all the same:

In [9]:
dataSea.columns=dataNY.columns=dataBos.columns

Now

In [10]:
set(dataSea.columns)&set(dataNY.columns)&set(dataBos.columns)

{'1940', '1970', '1990', '2010', '2020', 'Race'}

## Concatenating

In [11]:
# adding a column:

dataSea['City']='Seattle'
dataNY['City']='NY'
dataBos['City']='Boston'

Let's concatenate:

In [12]:
SeaNYBos=pd.concat([dataSea,dataNY,dataBos])
SeaNYBos

,Race,2020,2010,1990,1970,1940,City
0,White (non-Hispanic),59.5%,66.3%,73.7%,85.3%[c],NaN,Seattle
1,Asian (non-Hispanic),16.9%,13.7%,11.8%,4.2%,2.8%,Seattle
2,Hispanic or Latino,8.2%,6.6%,3.6%,2.0%[c],NaN,Seattle
3,Black or African American (non-Hispanic),6.8%,7.7%,10.1%,7.1%,1.0%,Seattle
4,Other (non-Hispanic),0.6%,0.2%,NaN,NaN,NaN,Seattle
5,Two or more races (non-Hispanic),7.3%,4.4%,NaN,NaN,NaN,Seattle
0,White (non-Hispanic),30.9%,33.3%,43.4%,64.0%,92.1%,NY
1,Hispanic or Latino,28.3%,28.6%,23.7%,15.2%,1.6%,NY
2,Black or African American (non-Hispanic),20.2%,22.8%,28.8%,21.1%,6.1%,NY
3,Asian and Pacific Islander (non-Hispanic),15.6%,12.6%,7.0%,1.2%,0.2%,NY


In [13]:
#standardizing the race column

changes={"White (non-Hispanic)":"White",
"Non-Hispanic Whites":"White",
"Asian (non-Hispanic)":"Asian",
"Asian and Pacific Islander (non-Hispanic)":"Asian",
"Hispanic or Latino (of any race)":"Hispanic or Latino",
"Black or African American (non-Hispanic)":"Black",
"Native American (non-Hispanic)":"Native American",
"Other (non-Hispanic)":"Other",
"Native American (non-Hispanic)":"Other",
"Native American":"Other",
"Two or more races (non-Hispanic)":"Two or more races"}

In [14]:
SeaNYBos.Race.replace(to_replace=changes,inplace=True)
SeaNYBos

,Race,2020,2010,1990,1970,1940,City
0,White,59.5%,66.3%,73.7%,85.3%[c],NaN,Seattle
1,Asian,16.9%,13.7%,11.8%,4.2%,2.8%,Seattle
2,Hispanic or Latino,8.2%,6.6%,3.6%,2.0%[c],NaN,Seattle
3,Black,6.8%,7.7%,10.1%,7.1%,1.0%,Seattle
4,Other,0.6%,0.2%,NaN,NaN,NaN,Seattle
5,Two or more races,7.3%,4.4%,NaN,NaN,NaN,Seattle
0,White,30.9%,33.3%,43.4%,64.0%,92.1%,NY
1,Hispanic or Latino,28.3%,28.6%,23.7%,15.2%,1.6%,NY
2,Black,20.2%,22.8%,28.8%,21.1%,6.1%,NY
3,Asian,15.6%,12.6%,7.0%,1.2%,0.2%,NY


In [15]:
#setting city and race as indices
SeaNYBos.set_index(['City', 'Race'],inplace=True)
SeaNYBos

2020   2010   1990      1970   1940
City    Race                                                    
Seattle White               59.5%  66.3%  73.7%  85.3%[c]    NaN
        Asian               16.9%  13.7%  11.8%      4.2%   2.8%
        Hispanic or Latino   8.2%   6.6%   3.6%   2.0%[c]    NaN
        Black                6.8%   7.7%  10.1%      7.1%   1.0%
        Other                0.6%   0.2%    NaN       NaN    NaN
        Two or more races    7.3%   4.4%    NaN       NaN    NaN
NY      White               30.9%  33.3%  43.4%     64.0%  92.1%
        Hispanic or Latino  28.3%  28.6%  23.7%     15.2%   1.6%
        Black               20.2%  22.8%  28.8%     21.1%   6.1%
        Asian               15.6%  12.6%   7.0%      1.2%   0.2%
        Other                0.2%   0.2%   0.4%      0.1%    NaN
        Two or more races    3.4%   1.8%    NaN       NaN    NaN
Boston  White               44.7%  47.0%  59.0%  79.5%[f]  96.6%
        Black               22.0%  24.4%  23.8%     16.3%   3.1%
        Hispanic or Latino  19.5%  17.5%  10.8%   2.8%[f]   0.1%
        Asian                9.7%   8.9%   5.3%      1.3%   0.2%
        Two or more races    3.2%   3.9%      –         –      –
        Other                0.2%   0.4%   0.3%      0.2%      –

In [16]:
SeaNYBos=SeaNYBos.replace('\[.+\]|%','', regex=True)
SeaNYBos

2020  2010  1990  1970  1940
City    Race                                            
Seattle White               59.5  66.3  73.7  85.3   NaN
        Asian               16.9  13.7  11.8   4.2   2.8
        Hispanic or Latino   8.2   6.6   3.6   2.0   NaN
        Black                6.8   7.7  10.1   7.1   1.0
        Other                0.6   0.2   NaN   NaN   NaN
        Two or more races    7.3   4.4   NaN   NaN   NaN
NY      White               30.9  33.3  43.4  64.0  92.1
        Hispanic or Latino  28.3  28.6  23.7  15.2   1.6
        Black               20.2  22.8  28.8  21.1   6.1
        Asian               15.6  12.6   7.0   1.2   0.2
        Other                0.2   0.2   0.4   0.1   NaN
        Two or more races    3.4   1.8   NaN   NaN   NaN
Boston  White               44.7  47.0  59.0  79.5  96.6
        Black               22.0  24.4  23.8  16.3   3.1
        Hispanic or Latino  19.5  17.5  10.8   2.8   0.1
        Asian                9.7   8.9   5.3   1.3   0.2
        Two or more races    3.2   3.9     –     –     –
        Other                0.2   0.4   0.3   0.2     –

In [17]:
import numpy as np

SeaNYBos = SeaNYBos.replace(r'–', np.nan, regex=True)
SeaNYBos

2020  2010  1990  1970  1940
City    Race                                            
Seattle White               59.5  66.3  73.7  85.3   NaN
        Asian               16.9  13.7  11.8   4.2   2.8
        Hispanic or Latino   8.2   6.6   3.6   2.0   NaN
        Black                6.8   7.7  10.1   7.1   1.0
        Other                0.6   0.2   NaN   NaN   NaN
        Two or more races    7.3   4.4   NaN   NaN   NaN
NY      White               30.9  33.3  43.4  64.0  92.1
        Hispanic or Latino  28.3  28.6  23.7  15.2   1.6
        Black               20.2  22.8  28.8  21.1   6.1
        Asian               15.6  12.6   7.0   1.2   0.2
        Other                0.2   0.2   0.4   0.1   NaN
        Two or more races    3.4   1.8   NaN   NaN   NaN
Boston  White               44.7  47.0  59.0  79.5  96.6
        Black               22.0  24.4  23.8  16.3   3.1
        Hispanic or Latino  19.5  17.5  10.8   2.8   0.1
        Asian                9.7   8.9   5.3   1.3   0.2
        Two or more races    3.2   3.9   NaN   NaN   NaN
        Other                0.2   0.4   0.3   0.2   NaN

In [18]:
SeaNYBos.drop(columns='1940',inplace=True)
SeaNYBos

2020  2010  1990  1970
City    Race                                      
Seattle White               59.5  66.3  73.7  85.3
        Asian               16.9  13.7  11.8   4.2
        Hispanic or Latino   8.2   6.6   3.6   2.0
        Black                6.8   7.7  10.1   7.1
        Other                0.6   0.2   NaN   NaN
        Two or more races    7.3   4.4   NaN   NaN
NY      White               30.9  33.3  43.4  64.0
        Hispanic or Latino  28.3  28.6  23.7  15.2
        Black               20.2  22.8  28.8  21.1
        Asian               15.6  12.6   7.0   1.2
        Other                0.2   0.2   0.4   0.1
        Two or more races    3.4   1.8   NaN   NaN
Boston  White               44.7  47.0  59.0  79.5
        Black               22.0  24.4  23.8  16.3
        Hispanic or Latino  19.5  17.5  10.8   2.8
        Asian                9.7   8.9   5.3   1.3
        Two or more races    3.2   3.9   NaN   NaN
        Other                0.2   0.4   0.3   0.2

Two columns back from index:

In [19]:

SeaNYBos.reset_index(drop=False, inplace=True)
SeaNYBos

,City,Race,2020,2010,1990,1970
0,Seattle,White,59.5,66.3,73.7,85.3
1,Seattle,Asian,16.9,13.7,11.8,4.2
2,Seattle,Hispanic or Latino,8.2,6.6,3.6,2.0
3,Seattle,Black,6.8,7.7,10.1,7.1
4,Seattle,Other,0.6,0.2,NaN,NaN
5,Seattle,Two or more races,7.3,4.4,NaN,NaN
6,NY,White,30.9,33.3,43.4,64.0
7,NY,Hispanic or Latino,28.3,28.6,23.7,15.2
8,NY,Black,20.2,22.8,28.8,21.1
9,NY,Asian,15.6,12.6,7.0,1.2


## Formatting

In [ ]:
SeaNYBos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   City    18 non-null     object
 1   Race    18 non-null     object
 2   2020    18 non-null     object
 3   2010    18 non-null     object
 4   1990    14 non-null     object
 5   1970    14 non-null     object
dtypes: object(6)
memory usage: 992.0+ bytes


toDo: turn these as numeric:

In [ ]:
SeaNYBos.iloc[:,2:]

,2020,2010,1990,1970
0,59.5,66.3,73.7,85.3
1,16.9,13.7,11.8,4.2
2,8.2,6.6,3.6,2.0
3,6.8,7.7,10.1,7.1
4,0.6,0.2,NaN,NaN
5,7.3,4.4,NaN,NaN
6,30.9,33.3,43.4,64.0
7,28.3,28.6,23.7,15.2
8,20.2,22.8,28.8,21.1
9,15.6,12.6,7.0,1.2


In [24]:
SeaNYBos['2020'] = SeaNYBos['2020'].apply(pd.to_numeric, errors='coerce')
SeaNYBos['2010'] = SeaNYBos['2010'].apply(pd.to_numeric, errors='coerce')
SeaNYBos['1990'] = SeaNYBos['1990'].apply(pd.to_numeric, errors='coerce')
SeaNYBos['1970'] = SeaNYBos['1970'].apply(pd.to_numeric, errors='coerce')
SeaNYBos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   City    18 non-null     object 
 1   Race    18 non-null     object 
 2   2020    18 non-null     float64
 3   2010    18 non-null     float64
 4   1990    14 non-null     float64
 5   1970    14 non-null     float64
dtypes: float64(4), object(2)
memory usage: 992.0+ bytes


ToDo: Turn this as categorical

In [27]:
SeaNYBos['Race'] = SeaNYBos.Race.astype('category')
SeaNYBos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   City    18 non-null     object  
 1   Race    18 non-null     category
 2   2020    18 non-null     float64 
 3   2010    18 non-null     float64 
 4   1990    14 non-null     float64 
 5   1970    14 non-null     float64 
dtypes: category(1), float64(4), object(1)
memory usage: 1.1+ KB


## Exporting:

ToDO

In [31]:
#save SeaNYBos for R!
r_SeaNYBos = pd.to_r_dataframe(SeaNYBos)

AttributeError: module 'pandas' has no attribute 'to_r_dataframe'